# Gym Ignition Example

## Installation

In [0]:
# Disable stdout (use output.show() to print it)
%%capture --no-stderr output

# INSTALL CMAKE, SWIG, NINJA
!\
    wget -O - https://apt.kitware.com/keys/kitware-archive-latest.asc 2>/dev/null | sudo apt-key add - &&\
    apt-add-repository 'deb https://apt.kitware.com/ubuntu/ bionic main' &&\
    apt-get -qq update &&\
    apt-get -qq install cmake swig ninja-build

# INSTALL IGNITION LIBRARIES
!\
    apt-get -qq update &&\
    echo "deb http://packages.osrfoundation.org/gazebo/ubuntu-stable `lsb_release -cs` main" > /etc/apt/sources.list.d/gazebo-stable.list &&\
    echo "deb http://packages.osrfoundation.org/gazebo/ubuntu-prerelease `lsb_release -cs` main" > /etc/apt/sources.list.d/gazebo-prerelease.list &&\
    echo "deb http://packages.osrfoundation.org/gazebo/ubuntu-nightly `lsb_release -cs` main" > /etc/apt/sources.list.d/gazebo-prerelease.list &&\
    wget http://packages.osrfoundation.org/gazebo.key -O - | apt-key add - &&\
    apt-get -qq update &&\
    apt-get -qq install -y --no-install-recommends \
        libgflags-dev \
        libignition-cmake2-dev \
        libignition-plugin-dev \
        libignition-math6-dev \
        libignition-common3-dev \
        libignition-transport7-dev \
        libignition-msgs4-dev \
        libignition-tools-dev \
        libignition-fuel-tools3-dev \
        libsdformat8-dev \
        libignition-physics-dev \
        libignition-rendering-dev \
        libignition-sensors2-dev \
        libignition-gui2-dev

In [0]:
# Disable stdout (use output.show() to print it)
%%capture --no-stderr output

# INSTALL IGNITION GAZEBO
!\
    cd /tmp &&\
    rm -rf ign-gazebo &&\
    git clone --depth 1 https://github.com/diegoferigo/ign-gazebo &&\
    cd ign-gazebo && mkdir -p build && cd build &&\
    cmake -DCMAKE_INSTALL_PREFIX=/usr -DCMAKE_BUILD_TYPE="Release" -DBUILD_TESTING:BOOL=OFF .. &&\
    cmake --build . --target install -- -j8 &&\
    rm -r /tmp/ign-gazebo

In [7]:
# INSTALL GYM IGNITION
# Note that we have to prepend the PATH variable in order to get the updated CMake
!PATH=/usr/bin:$PATH pip3 install gym-ignition

In [0]:
# CONFIGURE THE ENVIRONMENT
import os
import gym_ignition
import gym_ignition_data

data_path = gym_ignition_data.get_data_path()
module_path = gym_ignition.__path__[0]

os.environ['IGN_GAZEBO_SYSTEM_PLUGIN_PATH'] = f"{module_path}/plugins"
os.environ['IGN_GAZEBO_RESOURCE_PATH'] = f"{data_path}:{data_path}/worlds"

In [12]:
%%capture --no-stderr --no-stdout output

import os
import gym
import gym_ignition
import gym_ignition_data
from gym_ignition.utils import logger, resource_finder

# Add the search path to find the models and worlds
resource_finder.add_path_from_env_var("IGN_GAZEBO_RESOURCE_PATH")

# Set gym verbosity
logger.set_level(gym.logger.INFO)
# logger.set_level(gym.logger.DEBUG)

logger.info("Initializing the environment")

# Create the environment
# env = gym.make("CartPole-v1")
env = gym.make("CartPoleDiscrete-Gazebo-v0")
# env = gym.make("CartPoleDiscrete-PyBullet-v0")


# Initialize the seed
env.seed(42)

for epoch in range(10):
    # Reset the environment
    observation = env.reset()

    # Initialize returned values
    done = False
    total_reward = 0

    while not done:
        # Execute a random action
        action = env.action_space.sample()
        observation, reward, done, _ = env.step(action)

        # Render the environment
        # NOTE: You cannot render the environments in Colab
        # env.render('human')

        # Accumulate the reward
        total_reward += reward

        # Print the observation
        msg = ""
        for value in observation:
            msg += "\t%.6f" % value
        logger.debug(msg)

    logger.info("Total reward for episode #{}: {}".format(epoch, total_reward))

env.close()


INFO: Initializing the environment
INFO: Making new env: CartPoleDiscrete-Gazebo-v0


WARN: This runtime does not support setting the control mode


INFO: Total reward for episode #0: 384.0
INFO: Total reward for episode #1: 630.0
INFO: Total reward for episode #2: 413.0
INFO: Total reward for episode #3: 132.0
INFO: Total reward for episode #4: 189.0
INFO: Total reward for episode #5: 199.0
INFO: Total reward for episode #6: 226.0
INFO: Total reward for episode #7: 585.0
INFO: Total reward for episode #8: 534.0
INFO: Total reward for episode #9: 105.0
